In [ ]:
X_1 = pd.DataFrame()  # dataframe to hold new features
X_1["LivLotRatio"] = df.GrLivArea / df.LotArea

In [ ]:
# One-hot encode BldgType. Use `prefix="Bldg"` in `get_dummies`
X_2 = pd.get_dummies(df.BldgType, prefix="Bldg")
# Multiply
X_2 = X_2.mul(df.GrLivArea, axis=0)

In [ ]:
components = [ "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "Threeseasonporch",
               "ScreenPorch"]
X_3["PorchTypes"] = df[components].gt(0).sum(axis=1)

In [ ]:
X_4["MSClass"] = df.MSSubClass.str.split("_", n=1, expand=True)[0]

In [ ]:
X_5["MedNhbdArea"] = df.groupby("Neighborhood")["GrLivArea"].transform("median")